Let's store our final models in this notebook; through these models we can gauge the effectiveness of predictions via elveraging Kalshi's event contract exchange. Starting with Monte-Carlo.

In [47]:
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
from pandas.tseries.offsets import BDay
import random
import math
import numpy as np
from prophet import Prophet
from prophet.plot import plot_plotly, plot_components_plotly
from scipy.stats import norm
import plotly.graph_objects as go
import warnings

warnings.filterwarnings('ignore')

In [48]:
today = datetime.today().strftime('%Y-%m-%d')
start = '2016-01-01'

sp_df = yf.download('^GSPC', start, today)
sp_df.reset_index(inplace = True)

df = sp_df[['Date', 'Adj Close']]

# rename for prophet usage
df.rename(columns = {'Date' : 'ds', 'Adj Close' : 'y'}, inplace = True)

# quickly vizualizing data
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x = df['ds'], y = df['y']))

fig1.update_layout(
    title_text = "S&P 500 Pricing with Prophet"
)

# adding sliders and buttons for interaction
fig1.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(
                    count = 1,
                    label = '1m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 6,
                    label = '6m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 1,
                    label = 'YTD',
                    step = 'year',
                    stepmode = 'todate'
                ),
                dict(
                    count = 1,
                    label = '1y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(
                    count = 5,
                    label = '5y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(step = 'all')
            ])
        ),
        rangeslider = dict(
            visible =True
        ),
        type = 'date'
    )
)

fig1.show()

[*********************100%***********************]  1 of 1 completed


Now, below, we specifically have the Monte-Carlo Simulation (Euler Discretization).

In [51]:
data = pd.DataFrame()
data = sp_df['Adj Close']
log_returns = np.log(1 + data.pct_change())

#returns stats
u = log_returns.mean()
var = log_returns.var()
drift = u - (0.5 * var)
stdev = log_returns.std()

# trading days per year and num of sims
t_intervals = 252
iterations = 10

daily_returns = np.exp(drift + stdev * norm.ppf(np.random.rand(t_intervals, iterations)))

s0 = data.iloc[-1]

price_list = np.zeros_like(daily_returns)
price_list[0] = s0

for t in range(1, t_intervals):
    price_list[t] = price_list[t-1] * daily_returns[t]

price_list

columns = []
for i in range(0,iterations):
     columns.append("sim" + str(i))

# annual stdev of returns
stdev_a = log_returns.std() * 252 ** 0.5

# risk-free rate of return (bonds)
r = 0.026

# time horizon (years)
T = 1

# intervals within horizon (trading days)
t_intervals = 252
delta_t = T / t_intervals

Z = np.random.standard_normal((t_intervals + 1, iterations))
S = np.zeros_like(Z)
S0 = data.iloc[-1]
S[0] = S0

for t in range(1, t_intervals + 1):
    S[t] = S[t-1] * np.exp((r - 0.5 * stdev_a ** 2) * delta_t + stdev_a * delta_t ** 0.5 * Z[t])

df2 = pd.DataFrame(S, columns = columns)
df2['date'] = pd.date_range(start= today, periods=len(df2), freq=BDay())

# Let's visualize the Euler Discretizations
fig2 = go.Figure()
for name in columns:
    fig2.add_trace(go.Scatter(x = df2['date'], y = df2[name]))

fig2.update_layout(
    title_text = "S&P 500 Pricing with Euler Discretization"
)

# adding sliders and buttons for interaction
fig2.update_layout(
    xaxis = dict(
        rangeselector = dict(
            buttons = list([
                dict(
                    count = 1,
                    label = '1m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 6,
                    label = '6m',
                    step = 'month',
                    stepmode = 'backward'
                ),
                dict(
                    count = 1,
                    label = 'YTD',
                    step = 'year',
                    stepmode = 'todate'
                ),
                dict(
                    count = 1,
                    label = '1y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(
                    count = 5,
                    label = '5y',
                    step = 'year',
                    stepmode = 'backward'
                ),
                dict(step = 'all')
            ])
        ),
        rangeslider = dict(
            visible =True
        ),
        type = 'date'
    )
)

fig2.show()

Now, for out final prophet model, using the optimized parameters we found in the "prophet_opt" file and also using a 95% confidence interval.

In [50]:
# build prophet model for price forcasting
m = Prophet(daily_seasonality = False, seasonality_mode='multiplicative', changepoint_prior_scale=1000, seasonality_prior_scale=0.01, changepoint_range=0.85, interval_width=0.95)
for col in df.columns:
    if col not in ['ds', 'y']:
        m.add_regressor(col, mode = 'additive')
m.fit(df)

# here we create a year's worth of time series for the prophet model
future = m.make_future_dataframe(periods = 365)
business_days = BDay().onOffset
filter = pd.to_datetime(future['ds']).map(business_days)
future = future[filter]
forecast = m.predict(future)

plot_plotly(m, forecast)

INFO:cmdstanpy:start chain 1
INFO:cmdstanpy:finish chain 1


In reality, when speculating on Kalshi's S&P end of week price contract market, the results in a year are irrelvant to us, we only care about the next few weeks at most, meaning that the models only serves to predict the very near future. Any of these specific results can be garnered from the dataframes/np arrays we have created for these models.